In [1]:
import pickle, dill
import sys
import json
sys.path.append("..")
import experimenter
from experimenter.MultiTask.data import MultiTaskProvider
from experimenter.utils import utils as U
path = "/Users/jkhouja/workspace/experiments/moviecorpus/results/MOVIE_CORPUS/Jan_18_2021_16_05_15_752638_9993/"


In [17]:
config = json.load(open(path + "results.json", 'r'))

# Will load and process data
s = U.load_class(config['processor']['module'], config['processor']['class'], {'config':config, 'checkpoint' : path + "processor.pickle"}, pass_params_as_dict=True)

# Will override parameters
#s.load(path + "processor.pickle")

m = U.load_class(config['model']['module'], config['model']['class'], config)
m.load()
m.eval() #Set to inference mode

e = U.load_class(config['evaluator']['module'], config['evaluator']['class'],  config )

Loading processor files from /Users/jkhouja/workspace/experiments/moviecorpus/results/MOVIE_CORPUS/Jan_18_2021_16_05_15_752638_9993/processor.pickle0
Loading processor files from /Users/jkhouja/workspace/experiments/moviecorpus/results/MOVIE_CORPUS/Jan_18_2021_16_05_15_752638_9993/processor.pickle1


In [41]:
size = 4
dummy_data = {'inp': [size*['this product  of the Instagram is not a waste of money!']], 'label': [size * [['neg']]], 'mask': [size * [1]]}


In [33]:
dummy_data

{'inp': [['this {product_descriptor} of the Instagram is a waste of money!',
   'this {product_descriptor} of the Instagram is a waste of money!',
   'this {product_descriptor} of the Instagram is a waste of money!',
   'this {product_descriptor} of the Instagram is a waste of money!']],
 'label': [['neg', 'neg', 'neg', 'neg']],
 'mask': [[1, 1, 1, 1]]}

In [21]:
# predict one:
text = "great"

# To batchify
size = 4
dummy_data = {'inp': [size*[text]], 'label': [size * ['+']], 'mask': [size * [1]]}


data_as_batches = s(dummy_data, list_input=True, as_batches=True)
res = s.decode(s._from_batches([m(U.move_batch(batch, "cpu")) for batch in data_as_batches]), list_input=True)

res_one = {}
for k in res.keys():
    res_one[k] = res[k][0][0]

print(res_one['inp'], ":", res_one['pred'])

great : ['pos']


In [42]:
# Predict list
data = dummy_data # Test set
data_as_batches = s(data, list_input=True, as_batches=True)

res = s._from_batches([m(U.move_batch(batch, "cpu")) for batch in data_as_batches])
res = s.decode(res, list_input=True)
res

{'inp': [['this product  of the <UNK> is not a waste of money!',
   'this product  of the <UNK> is not a waste of money!',
   'this product  of the <UNK> is not a waste of money!',
   'this product  of the <UNK> is not a waste of money!']],
 'label': [[['neg'], ['neg'], ['neg'], ['neg']]],
 'mask': [[1, 1, 1, 1]],
 'out': [[[1.206682801246643, -1.2668569087982178],
   [1.206682801246643, -1.2668569087982178],
   [1.206682801246643, -1.2668569087982178],
   [1.206682801246643, -1.2668569087982178]]],
 'pred': [[['neg'], ['neg'], ['neg'], ['neg']]],
 'meta': [[[0.9222659468650818, 0.0777340829372406],
   [0.9222659468650818, 0.0777340829372406],
   [0.9222659468650818, 0.0777340829372406],
   [0.9222659468650818, 0.0777340829372406]]]}

In [37]:
max_idx = 4
for idx in range(max_idx):
    print(res['inp'][0][idx][-50:], res['pred'][0][idx], res['label'][0][idx])

this <UNK> of the <UNK> is a waste of money! ['neg'] ['neg']
this <UNK> of the <UNK> is a waste of money! ['neg'] ['neg']
this <UNK> of the <UNK> is a waste of money! ['neg'] ['neg']
this <UNK> of the <UNK> is a waste of money! ['neg'] ['neg']


In [38]:
# Evaluate
#e = U.load_class(config['evaluator']['module'], config['evaluator']['class'],  config )
avg_acc = [e.get_metrics(m(U.move_batch(b, "cpu")))[0] for b in data_as_batches]
import numpy as np
np.mean(avg_acc)

1.0